In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDOneClassSVM
from sklearn.neighbors import LocalOutlierFactor

# Part 0: The Checklist
- [ ] Get the data
- [ ] Do the One Class SVM, Isolation Tree, and Local Outlier Factor
- [ ] Clip the Target
- [ ] Arithmetic Mean


In [2]:
df = pd.read_csv('sample_submission.csv')
df

,id,target,O2_1,O2_2,O2_3,O2_4,O2_5,O2_6,O2_7,NH4_1,...,NO3_5,NO3_6,NO3_7,BOD5_1,BOD5_2,BOD5_3,BOD5_4,BOD5_5,BOD5_6,BOD5_7
0,0,8.59,7.500,9.000,9.545,9.265,8.110,8.430,7.150,0.180,...,4.950,1.730,1.800,4.800,3.150,10.665,10.465,16.645,5.750,10.37
1,1,9.10,13.533,40.900,8.770,9.265,6.015,10.070,7.150,1.107,...,20.050,9.530,7.695,4.550,6.950,2.040,5.200,5.725,2.950,2.23
2,2,8.21,3.710,5.420,8.770,9.265,4.550,10.070,7.150,0.020,...,4.580,3.025,3.960,4.935,4.950,4.725,6.075,6.750,3.500,3.17
3,3,8.39,8.700,8.100,9.500,9.200,5.200,8.670,6.670,0.280,...,8.450,2.070,1.730,6.300,4.700,3.500,6.200,8.670,2.900,7.37
4,4,8.07,8.050,8.650,7.960,9.265,3.290,10.070,7.150,0.360,...,2.020,1.730,0.760,4.800,4.970,3.950,2.800,8.400,3.500,3.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3495,8.08,6.250,8.300,7.795,9.265,5.690,8.555,6.335,0.565,...,14.575,0.636,1.640,4.235,4.100,2.800,3.950,7.695,3.540,2.50
3496,3496,8.09,6.630,6.630,8.370,7.600,0.636,8.430,7.150,1.300,...,4.580,1.730,1.800,4.900,3.150,2.040,6.075,8.415,2.155,2.90
3497,3497,9.95,8.367,8.433,8.770,6.170,5.800,10.400,7.200,0.430,...,20.050,1.440,1.800,4.867,4.833,4.725,4.950,8.400,6.625,4.20
3498,3498,9.52,10.000,6.630,9.545,9.265,3.290,8.980,2.310,0.300,...,20.050,9.530,7.695,10.700,5.400,4.725,3.300,6.750,6.625,5.00


In [3]:
SGDOneClassSVM?

Init signature:
SGDOneClassSVM(
    nu=0.5,
    fit_intercept=True,
    max_iter=1000,
    tol=0.001,
    shuffle=True,
    verbose=0,
    random_state=None,
    learning_rate='optimal',
    eta0=0.0,
    power_t=0.5,
    warm_start=False,
    average=False,
)
Docstring:     
Solves linear One-Class SVM using Stochastic Gradient Descent.

This implementation is meant to be used with a kernel approximation
technique (e.g. `sklearn.kernel_approximation.Nystroem`) to obtain results
similar to `sklearn.svm.OneClassSVM` which uses a Gaussian kernel by
default.

Read more in the :ref:`User Guide <sgd_online_one_class_svm>`.

.. versionadded:: 1.0

Parameters
----------
nu : float, default=0.5
    The nu parameter of the One Class SVM: an upper bound on the
    fraction of training errors and a lower bound of the fraction of
    support vectors. Should be in the interval (0, 1]. By default 0.5
    will be taken.

fit_intercept : bool, default=True
    Whether the intercept should be estimated

In [28]:
ocs = SGDOneClassSVM(0.5)

In [29]:
X = df.drop('target', axis=1)
y = df.target

In [30]:
ocs.fit(X)
ocs_pred = ocs.predict(X)

In [31]:
len(np.where(ocs_pred==-1)[0])

12

In [32]:
np.where(ocs_pred==-1)[0]

array([ 0,  2,  3,  4,  5,  7,  8,  9, 10, 11, 14, 15])

In [25]:
ocs_n = SGDOneClassSVM(0.5)
ocs_n.fit(df)
ocs_n_pred = ocs_n.predict(df)

In [26]:
np.where(ocs_n_pred==-1)[0]

array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 14, 15])

In [27]:
len(np.where(ocs_n_pred==-1)[0])

13

In [33]:
ocs_n_pred

array([-1,  1, -1, ...,  1,  1,  1], dtype=int32)

In [37]:
ocs_id = SGDOneClassSVM(0.3)
ocs_id.fit(df.drop('id', axis=1))
ocs_id_pred = ocs_id.predict(df.drop('id', axis=1))
np.where(ocs_id_pred==-1)[0]

array([  76,  133,  282,  463,  476,  510,  613,  797, 1054, 1375, 1502,
       2006, 2533, 2535, 2627, 2793, 3464])